In [2]:
# Install folium:- pip install folium
# Install pandas:- pip install pandas
# Install geocoder:- pip install geocoder

#imported required libraries
import folium
import pandas as pd
from geopy.geocoders import Nominatim
import json
from datetime import datetime

#access loginsample.json and create dataframe
with open("loginsample.json") as jsonfile:
    data = json.load(jsonfile)
df = pd.DataFrame(data["account_activity"])

#Added two columns for longitude and latitude for ploting map
df['longitude']=""
df['latitude']=""


# update longitude and latitude in dataframe by finding it through city available in the data using geocode
locator = Nominatim(user_agent="RBharmal")

for index, row in df.iterrows():
    
    information = locator.geocode(row['city'] + " " + row['region'] + " " + row['country'])
    df.at[index,"latitude"] = information.latitude
    df.at[index,"longitude"] = information.longitude

#create map from the dataframe using folium
user_map = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()],zoom_start=4,tiles = 'Stamen Terrain')
for index, row in df.iterrows():
    #convert timestamp to normal datetime format
    datetime = datetime.fromtimestamp(row['timestamp']);
    
    #popup data creation
    html=f"""
        <p>Facebook Activity</p>
        <ul>
            <li><b>Action:</b> {row['action']}</li>
            <li><b>Time:</b> {datetime}</li>
            <li><b>IP Address:</b> {row['ip_address']} </li>
            <li><b>Region:</b> {row['region']}</li>
        </ul>
        </p>
        """
    iframe = folium.IFrame(html=html, width=200, height=200)
    popup = folium.Popup(iframe, max_width=2650)
    folium.Marker(
        location=[df.loc[index,'latitude'], df.loc[index,'longitude']],
         popup=popup).add_to(user_map)
#view map
user_map